In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

path='data/APPENDED'
data_path='data'
ref_dataset= np.load(os.path.join(path,'refrigerator.dat.npy'))
print(ref_dataset[0])
ref_bins=4
ref_window_size=2401
input_shape=(2401, 1)

ref_dataset=ref_dataset[:,1:]
print(ref_dataset.shape)

bin_max_count=[0 for i in range(ref_bins)]
ref_clusters=[6.64953227,193.52627165,456.50688195]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import BatchNormalization

def load_cnn():
    
    model=Sequential()

    model.add(Conv1D(filters=8,kernel_size=5,input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))    
    model.add(Flatten())
    model.add(Dense(100,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3,activation='sigmoid'))

    from keras.optimizers import Adam
    
    learning_rate = 0.1
    decay_rate = 2
    momentum = 0.8
    opt= Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay_rate)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [ ]:
import collections
from keras.utils import np_utils
from matplotlib import pyplot as plt
from keras.utils import normalize

bin_dict=collections.Counter(ref_dataset[:,1])

ref_data=[]
ref_target=[]

model=load_cnn()
k=0
count=0
graph_no=0
key_min = min(bin_dict.keys(), key=(lambda k: bin_dict[k]))
ref_min_bin=bin_dict[key_min]
print(ref_min_bin)

epo=0
while(epo<10):

    for i in range(ref_dataset.shape[0]-ref_window_size-1-200000):
        ref_data.append(ref_dataset[i:i+ref_window_size,0])

        appliance=ref_dataset[i+ref_window_size-1,1]
        #print(appliance)

        total=np.sum(np.array([1/np.power(ref_clusters[j]-appliance,2) for j in range (len(ref_clusters))]))
        prob0=(1/np.power(ref_clusters[0]-appliance,2))/total
        prob1=(1/np.power(ref_clusters[1]-appliance,2))/total
        prob2=(1/np.power(ref_clusters[2]-appliance,2))/total
        #prob3=(1/np.power(ref_clusters[3]-appliance[i],2))/total    
        #print(appliance,[prob0,prob1,prob2],np.sum([prob0*[ref_clusters[0]],prob1*[ref_clusters[1]],prob2*[ref_clusters[2]]]))
        ref_target.append([prob0,prob1,prob2])

data=np.array(ref_data)/500.0
data = normalize(data, axis=1)
target=np.array(ref_target)
data=data.reshape(data.shape[0],data.shape[1],1)
print(target.shape,target[0])


history=model.fit(data[:data.shape[0]-100000,:],target[:data.shape[0]-100000,:],epochs=1) 

ref_data=[]
ref_target=[]
k=0
count=count+1


    #            ref_data=[]
    #            ref_target=[]        
    #            for i in range(10000):
    #                ref_data.append(ref_dataset[700000+i:700000+i+ref_window_size,0])

    #                appliance=ref_dataset[700000+i+ref_window_size-1,1]
                    #print(appliance)

    #                total=np.sum(np.array([1/np.power(ref_clusters[j]-appliance,2) for j in range (len(ref_clusters))]))
    #                prob0=(1/np.power(ref_clusters[0]-appliance,2))/total
    #                prob1=(1/np.power(ref_clusters[1]-appliance,2))/total
    #                prob2=(1/np.power(ref_clusters[2]-appliance,2))/total
                    #prob3=(1/np.power(ref_clusters[3]-appliance[i],2))/total    
                    #print(appliance,[prob0,prob1,prob2],np.sum([prob0*[ref_clusters[0]],prob1*[ref_clusters[1]],prob2*[ref_clusters[2]]]))

    #                ref_target.append([prob0,prob1,prob2])

    #            data=np.array(ref_data)/500.0
    #            data = normalize(data, axis=1)
    #            target=np.array(ref_target)
    #            data=data.reshape(data.shape[0],data.shape[1],1)

print(model.evaluate(data[data.shape[0]-100000:,:],target[data.shape[0]-100000:,:]))
results=model.predict(data[data.shape[0]-100000:,:])
plt.plot(np.argmax(target[data.shape[0]-100000:,:],axis=1),'b',label='ACTUAL')
plt.plot(np.argmax(results,axis=1),'r--',label='PREDICTED')
plt.legend()
plt.savefig('graphs_soft_association/'+str(epo)+str(graph_no)+'.jpg')
plt.close()

ref_data=[]
ref_target=[]
k=0
count=0
graph_no=graph_no+1
epo=epo+1

In [ ]:
np.argmax(test_target,axis=1).shape

In [ ]:
test_target.shape

In [ ]:
test_target

In [ ]:
np.argmax(test_target,axis=1)